In [1]:
import numpy as np
from itertools import product
import sys
sys.path.append('/home/samuel.varga/projects/deep_learning/')
from deep_learning.training_utils import read_rotation_results, load_hp_opt_results

In [2]:
results_dir = '/work/samuel.varga/projects/2to6_hr_severe_wx/DEEP_LEARNING/results/'
target_column = 'any_severe__36km'

In [3]:
combo_list = []
for p_s, lrate, cs, i, loss in product([0.01, 0.1, 0.25], [0.0001, 0.001, 0.01, 0.1], ([2,1,2,1],[2,2,2,2],[2,3,2,3],[4,3,2,2]), (1,2,3,4), ['binary_crossentropy']):
    #U-net architectural parameters
    conv_filters=[n*i for n in [32,64,128,256]]
    max_pool=[2,2,2,2]
    conv_layers =[{'filters': f, 'kernel_size': (s), 'pool_size': (p), 'strides': (p)} if p > 1
                       else {'filters': f, 'kernel_size': (s), 'pool_size': None, 'strides': None}
                       for s, f, p, in zip(cs, conv_filters, max_pool)]
    args={'lrate':lrate, 'loss':loss,'activation_conv':'relu','activation_out':'sigmoid',
         'p_spatial_dropout':p_s, 'filters':conv_filters, 'size':cs, 'pool':max_pool, 'shape':(16,16), 'target_column':target_column, 'i':i}
    fbase = f"{args['target_column']}_Rot_*_{args['shape'][0]}_{args['shape'][1]}_lrate_{args['lrate']}_spatial_dropout_{args['p_spatial_dropout']}_i_{args['i']}_filters_{args['filters']}_size_{args['size']}_pool_{args['pool']}_loss_{args['loss']}_results.pkl"
    combo_list.append(fbase)
    

In [4]:
results = load_hp_opt_results(results_dir, combo_list)

In [23]:
hp_results={}
for k in results.keys():
    temp = [x['predict_val_eval'] for x in results[k]]
    mean = {k : np.nanmean([z[k] for z in temp], axis=0) for k in temp[0].keys()}
    std = {k : np.nanstd([z[k] for z in temp], axis=0) for k in temp[0].keys()}
    hp_results[k] = {'mean':mean, 'std':std}

/tmp/ipykernel_1389192/3086440905.py:4: RuntimeWarning: Mean of empty slice
  mean = {k : np.nanmean([z[k] for z in temp], axis=0) for k in temp[0].keys()}


In [41]:
mean_hp = {k: hp_results[k]['mean'] for k in hp_results}

for metric in mean_hp[list(mean_hp.keys())[0]]:
    print(metric)
    Max = np.nanmax([mean_hp[k][metric] for k in mean_hp.keys()])
    Max_ind = np.nanargmax([mean_hp[k][metric] for k in mean_hp.keys()])
    Mean = np.nanmean([mean_hp[k][metric] for k in mean_hp.keys()])
    Min = np.nanmin([mean_hp[k][metric] for k in mean_hp.keys()])
    Min_ind = np.nanargmin([mean_hp[k][metric] for k in mean_hp.keys()])
    print(f'Max value at Loc. {Max_ind}: {Max}')
    print(f'Min value at Loc. {Min_ind}: {Min}')
    print(f'Mean value: {Mean}')

loss
Max value at Loc. 51: 0.23874521255493164
Min value at Loc. 81: 0.14839475005865096
Mean value: 0.17214585713911473
sparse_categorical_accuracy
Max value at Loc. 0: 0.9427213430404663
Min value at Loc. 0: 0.9427213430404663
Mean value: 0.9427213430404661
Brier score
Max value at Loc. 51: 0.06016220524907112
Min value at Loc. 57: 0.03960905596613884
Mean value: 0.04627631571339934
auc
Max value at Loc. 153: 0.9039740681648254
Min value at Loc. 111: 0.4946383714675903
Mean value: 0.7504971976081531
prc
Max value at Loc. 142: 0.409456342458725
Min value at Loc. 111: 0.05642695352435112
Mean value: 0.26010108756211897
FN
Max value at Loc. 44: 23025.2
Min value at Loc. 128: 4692.4
Mean value: 12902.757291666669
FP
Max value at Loc. 128: 63363.4
Min value at Loc. 44: 0.0
Mean value: 26263.879166666666
MAE
Max value at Loc. 46: 0.1320781320333481
Min value at Loc. 2: 0.08103374913334846
Mean value: 0.09519293502365288
TN
Max value at Loc. 44: 380430.8
Min value at Loc. 128: 317067.4
Mean

In [44]:
list(mean_hp.keys())[153]

'any_severe__36km_Rot_*_16_16_lrate_0.001_spatial_dropout_0.25_i_2_filters_[64, 128, 256, 512]_size_[2, 3, 2, 3]_pool_[2, 2, 2, 2]_loss_binary_crossentropy_results.pkl'

In [43]:
mean_hp[list(mean_hp.keys())[153]]

{'loss': 0.1489591896533966,
 'sparse_categorical_accuracy': 0.9427213430404663,
 'Brier score': 0.04221288859844208,
 'auc': 0.9039740681648254,
 'prc': 0.40022720098495485,
 'FN': 5739.4,
 'FP': 45851.4,
 'MAE': 0.08904152810573578,
 'TN': 334579.4,
 'TP': 17285.8,
 'bss': 0.9577871203422547,
 'csi': 0.1672684520483017}

In [45]:
list(mean_hp.keys())[67]

'any_severe__36km_Rot_*_16_16_lrate_0.0001_spatial_dropout_0.1_i_4_filters_[128, 256, 512, 1024]_size_[2, 1, 2, 1]_pool_[2, 2, 2, 2]_loss_binary_crossentropy_results.pkl'

In [42]:
mean_hp[list(mean_hp.keys())[67]]

{'loss': 0.15007832199335097,
 'sparse_categorical_accuracy': 0.9427213430404663,
 'Brier score': 0.04284626096487045,
 'auc': 0.9015020370483399,
 'prc': 0.39287262558937075,
 'FN': 6488.2,
 'FP': 42076.2,
 'MAE': 0.0862571731209755,
 'TN': 338354.6,
 'TP': 16537.0,
 'bss': 0.9571537375450134,
 'csi': 0.1762111932039261}